# Install Azure side dependencies

In [2]:
%%capture
!pip install azure-core
!pip install azure-storage-blob
!pip install fastapi

# Import packages

In [26]:
import pickle
from io import BytesIO
import h5py
import numpy as np

import tensorflow as tf
from keras.models import load_model
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Declare Constants

In [34]:
BLOB_STORAGE_CONNECTION_STRING = " "
BLOB_CONTAINER_NAME = " "
DEEP_LEARNING_MODEL_PATH = "iris.h5"

# Function to load uploaded model from azure

In [28]:
def loadBlobModel(
        filePath,
        connectionString,
        containerName
):
    blob_service_client = BlobServiceClient.from_connection_string(connectionString)

    blob_client = blob_service_client.get_blob_client(container=containerName, blob=filePath)

    blob_data = BytesIO()
    blob_client.download_blob().download_to_stream(blob_data)
    blob_data.seek(0)

    with h5py.File(blob_data, 'r') as f:
        loaded_model = tf.keras.models.load_model(f)

    print(f"Loaded model is {loaded_model.summary()}")

    return loaded_model

# Call the function to load the tensorflow model into the object instance of model

In [29]:
model = loadBlobModel(
    DEEP_LEARNING_MODEL_PATH,
    BLOB_STORAGE_CONNECTION_STRING,
    BLOB_CONTAINER_NAME
)

Loaded model is <keras.src.engine.sequential.Sequential object at 0x7af40b70c610>


# Use loaded model to predict the class

In [32]:
encodedClasses = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [33]:
input_values = np.array([[5.1, 3.5,	1.4, 0.2]])

predictions = model.predict(input_values)

print(encodedClasses[np.argmax(predictions)])

1/1 [==============================] - 0s 20ms/step
Iris-setosa
